In [18]:
import json
import os
import torch
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers.models import Transformer, Pooling
from datasets import Dataset

In [ ]:
# 1. 원래 모델과 토크나이저 로드
# model = torch.load("best_model_full.pt", map_location=torch.device('cpu'))  # 로컬 .pt 파일
model = AutoModel.from_pretrained("BAAI/bge-m3")
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

# 2. 모델을 SentenceTransformer로 래핑
# modules는 Transformer와 Pooling 레이어로 구성
transformer = Transformer(model_name_or_path="BAAI/bge-m3")
transformer.auto_model = model  # 로드한 모델로 교체
pooling = Pooling(model.config.hidden_size, pooling_mode="mean")  # bge-m3은 mean pooling 사용

st_model = SentenceTransformer(modules=[transformer, pooling])

# 3. 변환된 모델 저장
st_model.save("bge_m3_finetuned_local")
print("모델이 'bge_m3_finetuned_local'에 저장되었습니다.")

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

모델이 'bge_m3_finetuned_local'에 저장되었습니다.


In [8]:
local_model_path = "bge_m3_finetuned_local/"

try:
    # AutoModel을 사용하여 로컬 모델을 불러옵니다.
    # model = AutoModel.from_pretrained(local_model_path)
    # print(f"'{local_model_path}'에서 모델을 성공적으로 불러왔습니다.")

    transformer = Transformer(model_name_or_path=local_model_path)
    transformer.auto_model = model  # 로드한 모델로 교체
    pooling = Pooling(model.config.hidden_size, pooling_mode="mean")  # bge-m3은 mean pooling 사용

    st_model = SentenceTransformer(modules=[transformer, pooling])
    
    print(f"'{local_model_path}'에서 모델을 성공적으로 불러왔습니다.")

except Exception as e:
    print(f"모델을 불러오는 중 오류가 발생했습니다: {e}")
    print("경로가 정확한지, 모델 파일이 손상되지 않았는지 확인해주세요.")


'bge_m3_finetuned_local/'에서 모델을 성공적으로 불러왔습니다.


In [9]:
from transformers import AutoModel, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset  # Hugging Face Datasets 라이브러리
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader  # 직접 데이터 로딩 시 사용
import json
import os
import random

# 1. 모델 로드
local_model_path = "bge_m3_finetuned_local/"

try:
    # AutoModel을 사용하여 로컬 모델을 불러옵니다.
    model = AutoModel.from_pretrained(local_model_path)
    print(f"'{local_model_path}'에서 모델을 성공적으로 불러왔습니다.")

except Exception as e:
    print(f"모델을 불러오는 중 오류가 발생했습니다: {e}")
    print("경로가 정확한지, 모델 파일이 손상되지 않았는지 확인해주세요.")
    exit() # 모델 로드 실패 시 프로그램 종료

tokenizer = AutoTokenizer.from_pretrained(local_model_path) # 토크나이저도 로컬에서 로드

# 2. 데이터 로드 및 전처리

# JSON 파일들이 있는 디렉토리 또는 파일 경로 리스트
data_dir = "dataset/"  # 실제 JSON 파일들이 있는 디렉토리 경로로 변경

# 데이터 로드 함수 (여러 JSON 파일 처리)
def load_multiple_data(data_path):
    combined_data = []
    if isinstance(data_path, str):
        print(f"데이터 디렉토리: {data_path}")
        for filename in os.listdir(data_path):
            if filename.endswith(".json"):
                file_path = os.path.join(data_path, filename)
                print(f"로드할 파일: {file_path}")
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    print(f"로드된 데이터 일부 (첫 번째 항목): {data[0] if data else None}")
                    combined_data.extend(data)
    elif isinstance(data_path, list):
        print(f"데이터 파일 리스트: {data_path}")
        for file_path in data_path:
            print(f"로드할 파일: {file_path}")
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                print(f"로드된 데이터 일부 (첫 번째 항목): {data[0] if data else None}")
                combined_data.extend(data)
    print(f"총 로드된 데이터 항목 수: {len(combined_data)}")
    return combined_data

# JSON 파일들을 로드합니다.
if isinstance(data_dir, str):
    data = load_multiple_data(data_dir)
else:
    data = load_multiple_data(data_dir)

train_examples = []
print("긍정 및 부정 샘플 생성 시작...")
for i, item in enumerate(data):
    english = item.get("English")
    korean = item.get("Korean")
    etoK = item.get("EtoK")
    KtoE = item.get("KtoE")

    if english and korean:
        train_examples.append({"text": [english, korean], "label": 1}) # 긍정 (같은 의미)
        # 부정 샘플 (다른 의미의 문장과 페어링) - 간단한 예시
        negative_item = random.choice(data)
        negative_english = negative_item.get("English")
        if negative_english and negative_english != english:
            train_examples.append({"text": [english, negative_english], "label": 0})

    if korean and etoK:
        train_examples.append({"text": [korean, etoK], "label": 1}) # 긍정
        negative_item = random.choice(data)
        negative_english = negative_item.get("English")
        if negative_english and negative_english != korean:
            train_examples.append({"text": [korean, negative_english], "label": 0})

    if english and KtoE:
        train_examples.append({"text": [english, KtoE], "label": 1}) # 긍정
        negative_item = random.choice(data)
        negative_korean = negative_item.get("Korean")
        if negative_korean and negative_korean != english:
            train_examples.append({"text": [english, negative_korean], "label": 0})

    if i < 5: # 처음 몇 개 샘플만 출력하여 확인
        print(f"생성된 학습 예시 {i}: {train_examples[-1] if train_examples else None}")

print(f"총 생성된 학습 예시 수: {len(train_examples)}")

# 데이터셋 생성
train_dataset = Dataset.from_dict({
    "text": [item["text"] for item in train_examples],
    "labels": [item["label"] for item in train_examples]
})
print("Hugging Face Dataset 생성 완료.")
print(f"Dataset 예시 (첫 번째 항목): {train_dataset[0] if train_dataset else None}")

# 토큰화 함수
def tokenize_function(examples):
    print("토큰화 함수 호출됨.")
    result = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
    print(f"토큰화 결과 (일부): {result['input_ids'][:2] if result['input_ids'] else None}")
    return result

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
print("데이터셋 토큰화 완료.")
# print(f"토큰화된 Dataset 예시 (첫 번째 항목): {tokenized_train_dataset[0] if tokenized_train_dataset else None}")

# 3. 파인튜닝

# 학습 하이퍼파라미터 설정
training_args = TrainingArguments(
    output_dir="./bge-m3-finetuned-transformers",
    per_device_train_batch_size=16,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    # evaluation_strategy="epoch", # 필요하다면 평가 데이터셋 준비
    save_strategy="epoch",
    logging_dir="./logs_transformers",
    report_to="none",
    label_names=["labels"]
)
print("TrainingArguments 설정 완료.")
print(f"TrainingArguments: {training_args}")

# 모델을 위한 사용자 정의 Trainer (손실 함수 변경)
class ContrastiveTrainer(Trainer):
    def compute_loss(self, model, inputs, num_items_in_batch=None, return_outputs=False):
        print(f"compute_loss inputs 키: {inputs.keys()}")
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]
        labels = inputs["labels"]
        batch_size = input_ids.size(0)

        # input_ids와 attention_mask는 (batch_size, seq_length) 형태일 것입니다.
        # 이진 분류를 위해 두 문장을 하나로 합쳐서 토큰화했을 가능성을 고려하여 처리합니다.
        # 또는 데이터셋 생성 시 이미 [text1, text2] 형태로 되어 있다면 아래와 같이 분리할 수 있습니다.
        # (batch_size, 2, seq_length) 형태라고 가정하고 분리합니다.
        if len(input_ids.shape) > 2 and input_ids.shape[1] == 2:
            input_ids_1 = input_ids[:, 0, :]
            attention_mask_1 = attention_mask[:, 0, :]
            input_ids_2 = input_ids[:, 1, :]
            attention_mask_2 = attention_mask[:, 1, :]
        else:
            # 두 문장이 합쳐져서 토큰화된 경우, 적절한 분리 로직이 필요합니다.
            # 이 예시에서는 간단하게 처리하거나, 데이터셋 생성 방식을 수정하는 것을 고려합니다.
            raise ValueError("입력 형태가 예상과 다릅니다. 데이터셋 생성 및 토큰화 과정을 확인하세요.")

        # 두 개의 텍스트를 각각 인코딩
        try:
            outputs1 = model(input_ids_1, attention_mask=attention_mask_1)
            embeddings1 = outputs1.last_hidden_state.mean(dim=1) # 또는 다른 풀링 방식

            outputs2 = model(input_ids_2, attention_mask=attention_mask_2)
            embeddings2 = outputs2.last_hidden_state.mean(dim=1) # 또는 다른 풀링 방식

            # 코사인 유사도 계산
            cos_sim = torch.nn.functional.cosine_similarity(embeddings1, embeddings2)

            # 목표: 같은 의미면 유사도 높이기 (label 1), 다른 의미면 유사도 낮추기 (label 0)
            # 간단한 이진 분류 손실 함수 (조정 필요)
            loss_fn = torch.nn.BCEWithLogitsLoss()
            logits = cos_sim # 코사인 유사도를 로짓으로 사용 (적절한 스케일링 필요할 수 있음)
            loss = loss_fn(logits.unsqueeze(1), labels.float().unsqueeze(1))

            print(f"배치 손실: {loss.item()}")
            return loss
        except Exception as e:
            print(f"compute_loss 오류 발생: {e}")
            print(f"입력 input_ids 형태: {input_ids.shape}")
            print(f"입력 attention_mask 형태: {attention_mask.shape}")
            print(f"입력 labels 형태: {labels.shape}")
            raise

# Trainer 객체 생성
trainer = ContrastiveTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset
)
print("Trainer 객체 생성 완료.")

# 모델 파인튜닝 시작
print("모델 파인튜닝 시작...")
trainer.train()

print("모델 파인튜닝 완료!")

# 4. 결과 확인

# (선택 사항) 파인튜닝된 모델 저장
trainer.save_model("./bge-m3-finetuned-transformers-best")
tokenizer.save_pretrained("./bge-m3-finetuned-transformers-best")
print("최고 성능 모델 및 토크나이저가 './bge-m3-finetuned-transformers-best'에 저장되었습니다.")

# (선택 사항) 추론 또는 임베딩 생성에 파인튜닝된 모델 활용
def get_embeddings(texts, model, tokenizer):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.cpu().numpy()

if data:
    sample_item = data[0]
    english_text = sample_item.get("English")
    korean_text = sample_item.get("Korean")
    etoK_text = sample_item.get("EtoK")

    if english_text and korean_text and etoK_text:
        finetuned_embeddings_en = get_embeddings([english_text], trainer.model, tokenizer)
        finetuned_embeddings_ko = get_embeddings([korean_text], trainer.model, tokenizer)
        finetuned_embeddings_ek = get_embeddings([etoK_text], trainer.model, tokenizer)

        print("\n--- 파인튜닝 후 임베딩 (예시) ---")
        print(f"영어 임베딩 (처음 5개): {finetuned_embeddings_en.flatten()[:5]}")
        print(f"한국어 임베딩 (처음 5개): {finetuned_embeddings_ko.flatten()[:5]}")
        print(f"EtoK 임베딩 (처음 5개): {finetuned_embeddings_ek.flatten()[:5]}")

# 추가적인 평가를 위해서는 별도의 평가 데이터셋과 metric을 정의해야 합니다.

'bge_m3_finetuned_local/'에서 모델을 성공적으로 불러왔습니다.
데이터 디렉토리: dataset/
로드할 파일: dataset/code-switch.json
로드된 데이터 일부 (첫 번째 항목): {'EtoK': 'Could you explain how 양자 컴퓨팅 and 양자 얽힘 make it possible to solve certain problems faster than classical methods?', 'KtoE': '고전적 방식보다 더 효율적인 quantum computing과 quantum entanglement의 핵심 원리는 무엇인가요?', 'English': 'Could you explain how quantum computing and quantum entanglement make it possible to solve certain problems faster than classical methods?', 'Korean': '양자 컴퓨팅과 양자 얽힘이 일부 문제를 고전적 방법보다 더 빠르게 해결하도록 만드는 핵심 원리는 무엇인가요?'}
로드할 파일: dataset/code-switch1.json
로드된 데이터 일부 (첫 번째 항목): {'topic': 'science/technology', 'type': 'fact', 'complexity': 'simple', 'contents': 'brief', 'EtoK': 'The 원자 is the basic unit of matter.', 'KtoE': '원자는 matter의 기본 단위이다.', 'English': 'The atom is the basic unit of matter.', 'Korean': '원자는 물질의 기본 단위이다.'}
로드할 파일: dataset/code-switch2.json
로드된 데이터 일부 (첫 번째 항목): {'topic': 'science/technology', 'type': 'fact', 'complexity': 'simple', 'content

Map:   0%|          | 0/1613 [00:00<?, ? examples/s]

토큰화 함수 호출됨.
토큰화 결과 (일부): [[0, 191147, 398, 73342, 3642, 110436, 242122, 136, 110436, 22, 14525, 19929, 3249, 442, 7722, 47, 86869, 24233, 44402, 4271, 56, 3501, 54704, 289, 150624, 32, 2, 2, 16837, 2268, 6, 93721, 201692, 33994, 1291, 16837, 2268, 6, 249806, 238153, 469, 55245, 138153, 6086, 3776, 3761, 29152, 21491, 6116, 147660, 63054, 109690, 107900, 119199, 10380, 40771, 227539, 5144, 32, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 191147, 398, 73342, 3642, 110436, 242122, 136, 110436, 22, 14525, 19929, 3249, 442, 7722, 47, 86869, 24233, 44402, 4271, 56, 3501, 54704, 289, 150624, 32, 2, 2, 4865, 186768, 19732, 14602, 25134, 26320, 25, 7, 43454, 202, 10013, 159202, 67, 32, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

ValueError: 입력 형태가 예상과 다릅니다. 데이터셋 생성 및 토큰화 과정을 확인하세요.

In [8]:
print(tokenized_train_dataset[0].keys())
print(tokenized_train_dataset.keys())

dict_keys(['text', 'labels', 'input_ids', 'attention_mask'])


AttributeError: 'Dataset' object has no attribute 'keys'